In [147]:
import nltk
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from keras.layers import Dense, Embedding, LSTM
from keras import Sequential


In [148]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaggod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\yaggod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yaggod\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [164]:
input_filepath = "task3_input.txt"
output_filepath = 'task3_output.txt'

with open(input_filepath, 'r', encoding='utf-8') as file:
    text = [file.read().lower()]

In [150]:
stopwords.words('russian')#.extend(list(punctuation) + ['…', '—', '.', '``'])

vectorizer = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.word_tokenize, stop_words=stopwords.words('russian'))
vectorizer.fit_transform(text)
vocabulary = vectorizer.vocabulary_

token_to_word_map = {}
word_to_token_map =  {}

for token, index in vocabulary.items(): 
    token_to_word_map[index + 1] = token
    word_to_token_map[token] = index + 1 
    # zero index is ignored

tokenized_words = nltk.word_tokenize(text[0], 'russian')


c:\Users\yaggod\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [151]:
tokens_sequence = []
for word in tokenized_words:
    token = word_to_token_map.get(word, 0)  # zero is ignored 
    if(not token == 0):
        tokens_sequence.append(token)

print(tokens_sequence)
print([token_to_word_map[token] for token in tokens_sequence])

[107, 929, 867, 4861, 1840, 5163, 1034, 4469, 1035, 2004, 2875, 149, 4984, 6, 4905, 1721, 2119, 9, 4356, 2876, 6, 1189, 9, 1997, 6, 4180, 9, 111, 4464, 2218, 6, 4031, 4985, 6, 1353, 1147, 9, 107, 128, 5128, 9, 3193, 4100, 6, 4045, 4043, 9, 2396, 1095, 6, 2207, 978, 6, 2988, 2165, 6, 2261, 1291, 5343, 873, 9, 1811, 4749, 1805, 6, 1326, 6, 5259, 1186, 9, 918, 6, 919, 5343, 5086, 9, 132, 133, 1424, 9, 4451, 5049, 6, 4000, 4985, 9, 760, 88, 760, 71, 3847, 9, 2961, 2495, 838, 4175, 9, 2179, 893, 691, 3732, 1946, 3770, 9, 2859, 4985, 9, 1031, 2239, 1147, 4985, 9, 1031, 78, 5332, 11, 895, 6, 286, 5125, 6, 4393, 995, 4367, 4354, 6, 2811, 4216, 5008, 4997, 6, 1870, 4882, 9, 1997, 4412, 1848, 4381, 6, 3960, 1893, 9, 4, 4947, 6, 1056, 309, 9, 5, 1997, 4, 3618, 6, 676, 1893, 6, 2668, 5, 3964, 1, 10, 151, 1, 4124, 2572, 2156, 1, 707, 3595, 4372, 5343, 2799, 9, 98, 1316, 1200, 99, 9, 5343, 2583, 787, 787, 6, 4378, 6, 3181, 4103, 4522, 9, 477, 6, 1056, 1, 10, 4096, 1655, 4, 4527, 4351, 9, 5, 885, 6, 

In [152]:
subsequence_length = 50

X = []
y = []

for i in range(len(tokens_sequence) - subsequence_length):
        X.append(tokens_sequence[i:i + subsequence_length])
        y.append(tokens_sequence[i + subsequence_length])

X = np.array(X)
y = np.array(y)

In [153]:
model = Sequential()
embedding_length = 100
lstm_units = 100

model.add(Embedding(output_dim = subsequence_length,
                    input_dim = len(vocabulary) + 1,
                    mask_zero = True))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(LSTM(lstm_units))
model.add(Dense(len(vocabulary) + 1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [154]:
model.fit(X, y)

498/498 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 7.0540


In [162]:
tokens_to_generate = 50

new_data = tokens_sequence[:subsequence_length]

for i in range(tokens_to_generate):
    subsequence = np.array([new_data[i: : subsequence_length + i]])
    predicted_probabilities = model.predict(subsequence, verbose=0)[0] 
    new_token = np.random.choice(len(predicted_probabilities), p = predicted_probabilities)
    new_data.append(new_token)

print(new_data)

[107, 929, 867, 4861, 1840, 5163, 1034, 4469, 1035, 2004, 2875, 149, 4984, 6, 4905, 1721, 2119, 9, 4356, 2876, 6, 1189, 9, 1997, 6, 4180, 9, 111, 4464, 2218, 6, 4031, 4985, 6, 1353, 1147, 9, 107, 128, 5128, 9, 3193, 4100, 6, 4045, 4043, 9, 2396, 1095, 6, 38, 1226, 2970, 1257, 2467, 1151, 2401, 5094, 1975, 1336, 3047, 3721, 1792, 446, 1669, 2905, 1901, 3469, 2308, 510, 1578, 781, 4079, 2112, 1584, 3906, 1985, 2183, 4785, 4076, 2564, 393, 5185, 3717, 3024, 5284, 2479, 4511, 2644, 4432, 5186, 2104, 1085, 1914, 1337, 4267, 2570, 2908, 246, 2238]


In [ ]:
words = []
for token in new_data:
    words.append(token_to_word_map.get(token, ''))
    
with open(output_filepath, 'w', encoding='utf-8') as file:
    file.write(' '.join(words))

['александр', 'грибоедов', 'горе', 'ума', 'комедия', 'четырёх', 'действиях', 'стихах', 'действующие', 'лица', 'павел', 'афанасьевич', 'фамусов', ',', 'управляющий', 'казённом', 'месте', '.', 'софья', 'павловна', ',', 'дочь', '.', 'лизанька', ',', 'служанка', '.', 'алексей', 'степанович', 'молчалин', ',', 'секретарь', 'фамусова', ',', 'живущий', 'доме', '.', 'александр', 'андреевич', 'чацкий', '.', 'полковник', 'скалозуб', ',', 'сергей', 'сергеевич', '.', 'наталья', 'дмитриевна', ',', '28', 'думаешь', 'пикетом', 'дыханье', 'незначащая', 'домосед', 'научат', 'царю', 'лекарства', 'женский', 'повыспрошу', 'ради', 'клубе', 'ветерком', 'ирритация', 'певец', 'кричали', 'прийти', 'наградил', 'виновата', 'зови', 'гимназий', 'силу', 'меньше', 'золоту', 'ругай', 'летам', 'могилу', 'убейся', 'сил', 'носится', 'везёт', 'члены', 'равный', 'побудь', 'шутит', 'немножко', 'страшит', 'обучений', 'старалась', 'чрезвычайно', 'матушка', 'директору', 'кружево', 'жену', 'смущаться', 'ночлег', 'пелен', 'благо